# 线性回归
## 两个关键点
### 损失函数
常选取一个非负数作为误差，比如平方函数，在线性系统中评估索引为 $i$ 的样本误差的表达式为
$$
l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2,
$$
小批量样本$n$的平均样本误差为
$$
L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2.
$$
### 优化函数 - 随机梯度下降
小批量随机梯度下降（mini-batch stochastic gradient descent）：先选取一组模型参数的初始值，如随机选取；接下来对参数进行多次迭代，使每次迭代都可能降低损失函数的值。在每次迭代中，先随机均匀采样一个由固定数目训练数据样本所组成的小批量（mini-batch）$\mathcal{B}$，然后求小批量中数据样本的平均损失有关模型参数的导数（梯度），最后用此结果与预先设定的一个正数的乘积作为模型参数在本次迭代的减小量。   
$$
(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b)
$$  
学习率: $\eta$代表在每次优化中，能够学习的步长的大小    
批量大小: $\mathcal{B}$是小批量计算中的批量大小batch size   

优化步骤：
- (i)初始化模型参数，一般来说使用随机初始化；
- (ii)我们在数据上迭代多次，通过在负梯度方向移动参数来更新每个参数。

## 线性回归模型从零开始的实现



In [4]:
# 引入模块
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

# 生成数据集
# 设置输入特征数 
num_inputs = 2
# 设置样本数
num_examples = 1000

# 设置真实权重和偏差值，以取得真实标签
true_w = [2, -3.4]
true_b = 4.2

features = torch.randn(num_examples, num_inputs,
                      dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float32)

#显示样本分布
#plt.scatter(features[:, 1].numpy(), labels.numpy(), 1);

#读取数据集
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 随机打乱样本
    random.shuffle(indices)  
    for i in range(0, num_examples, batch_size):
        # 考虑最后剩余数是否满足
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) 
        yield  features.index_select(0, j), labels.index_select(0, j)

batch_size = 10

#初始化模型参数
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)
#引入参数的梯度值
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

#定义模型
def linreg(X, w, b):
    return torch.mm(X, w) + b

#定义损失函数
def squared_loss(y_hat, y): 
    return (y_hat - y.view(y_hat.size())) ** 2 / 2
    
#定义优化函数
def sgd(params, lr, batch_size): 
    for param in params:
        param.data -= lr * param.grad / batch_size # 使用.data来更改参数，不改变梯度
        
#训练模型
# 学习率及迭代次数初始化
lr = 0.03
num_epochs = 5

net = linreg
loss = squared_loss

# 训练
for epoch in range(num_epochs):  
    
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  
        # 计算批量样本损失的梯度
        l.backward()  
        # 使用小批量随机梯度下降迭代模型参数
        sgd([w, b], lr, batch_size)  
        # 初始化梯度
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))
    
w, true_w, b, true_b

epoch 1, loss 0.038264
epoch 2, loss 0.000150
epoch 3, loss 0.000051
epoch 4, loss 0.000050
epoch 5, loss 0.000050


(tensor([[ 1.9998],
         [-3.3998]], requires_grad=True),
 [2, -3.4],
 tensor([4.2003], requires_grad=True),
 4.2)

## 线性回归模型使用pytorch的简洁实现


In [5]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1) 

torch.set_default_tensor_type('torch.FloatTensor')  #修改默认的tensor类型

#生成数据集
num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

#读取数据集
import torch.utils.data as Data

batch_size = 10

dataset = Data.TensorDataset(features, labels)

data_iter = Data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,  
    shuffle=True,              
    num_workers=2,  
)

#定义模型
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()   
        # 函数原型: `torch.nn.Linear(in_features, out_features, bias=True)`
        self.linear = nn.Linear(n_feature, 1) 

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)

net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    )

net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
        ]))

print(net)
print(net[0])

#初始化模型参数
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 等价于`net[0].bias.data.fill_(0)`

for param in net.parameters():
    print(param)
    
#定义损失函数
loss = nn.MSELoss()    
# function prototype: `torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')`

#定义优化函数
import torch.optim as optim
# 函数原型: `torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)`
optimizer = optim.SGD(net.parameters(), lr=0.03)  
print(optimizer) 

#训练
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # reset gradient, equal to net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))
    
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)
Parameter containing:
tensor([[-0.0152,  0.0038]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)
epoch 1, loss: 0.000248
epoch 2, loss: 0.000124
epoch 3, loss: 0.000138
[2, -3.4] tensor([[ 2.0009, -3.3998]])
4.2 tensor([4.2005])
